# Mixed language programming: `instant` and `ctypes`

Simon Funke, Hans Petter Langtangen, Joakim Sundnes, Ola Skavhaug

Date: **Oct 5, 2016**

## Avoiding wrapper code with *ctypes* and *instant*
Extension modules in Python have two main purposes:

1. Calling existing libraries inside Python programs.
2. Speeding up loops in Python.

Manual or Swig-generated wrapper code handles both cases, but for
simple tasks there are shortcuts:

* `ctypes` allows easy access to C datatypes and libraries
* `instant` enables inlining C and C++ function in Python programs

# The `instant` module

## `Instant` allows inlining of C and C++ functions in Python codes

* Install with 
```bash
pip install --user https://bitbucket.org/fenics-project/instant/get/master.zip
```
* The hello world example:

In [3]:
from instant import inline
source = """
double hw1(double r1, double r2)
{
return sin(r1 + r2);
}
"""
hw1 = inline(source)

x= 1.0
y =2.5
print("sin({0}+{1}) = {2}".format(x,y,hw1(x,y)))

sin(1.0+2.5) = -0.35078322768961984


## How it works

* C/C++ code is automatically wrapped and compiled at run-time
* Resulting object files are stored, only recompiled when source   code is changed
* Simple to use, but only works for smaller codes

## `Instant` also supports numpy arrays

In [5]:
import numpy                                                                                                                                                                                   
from instant import inline_with_numpy                                                                                                                                                          
                                                                                                                                                                                               
# Example 2: two array, both inout and of same size                                                                                                                                            
# Cannot avoid specifying all dimensions for both arrays                                                                                                                                       
c_code = """                                                                                                                                                                                   
double sum(int x1, int y1, double* array1, int x2, int y2, double* array2){                                                                                                                   
  double tmp = 0.0;                                                                                                                                                                            
  for (int i=0; i<x1; i++)                                                                                                                                                                     
    for (int j=0; j<y1; j++){                                                                                                                                                                  
      tmp = array1[i*y1 + j];                                                                                                                                                                  
      array1[i*y1 + j] = array2[i*y1 + j];                                                                                                                                                     
      array2[i*y1 + j] = tmp;                                                                                                                                                                  
    }                                                                                                                                                                                          
  return tmp;                                                                                                                                                                                  
}                                                                                                                                                                                              
"""                                                                                                                                                                                            
                                                                                                                                                                                               
sum_func = inline_with_numpy(c_code, arrays = [['x1', 'y1', 'array1'],                                                                                                                         
                                               ['x2', 'y2', 'array2']],                                                                                                                        
                             cache_dir="test_ex2_cache")                                                                                                                                       
                                                                                                                                                                                               
a = numpy.ones(4)                                                                                                                                                                              
a.shape = (2, 2)                                                                                                                                                                               
b = a.copy()                                                                                                                                                                                   
a *= 2                                                                                                                                                                                         
print(sum_func(a, b))  

--- Instant: compiling ---


2.0


# The `ctypes` module

##  `ctypes` gives access to C datatypes from  Python

* C libraries can be imported and called from python
* Work only in Python - no need for writing   wrapper code in C
* Less elegant interface than a regular Python extension module

## Basic usage
* Primitive C data types are interfaced:

```python
c_int, c_bool, c_double, c_char ...
```

* Libraries may be loaded by instantiating `ctypes.CDLL`:

```python
clib = ctypes.CDLL('./clib.so')
```

## Basic usage (2)

* Arguments and return type for library functions must be converted to the correct C type:

In [ ]:
c_arg1 = c_double(arg1)
c_arg2 = c_double(arg2)
clib.function1.restype = c_double

* After the conversion, calling library functions is intuitive:

In [ ]:
result = clib.function1(c_arg1,c_arg2)

## Hello world with `ctypes` (1)

Unmodified C code, compiled to a shared library;

```C
double hw1(double r1, double r2)
{
    double s;
    s = sin(r1 + r2);
    return s;
}
        
void hw2(double r1, double r2)
{
    double s;
    s = sin(r1 + r2);
    printf("Hello, World! sin(%g+%g)=%g\n", r1, r2, s);
}
        
/* special version of hw1 where the result is an argument: */
void hw3(double r1, double r2, double *s)
{
    *s = sin(r1 + r2);
}
```

## Hello world with ctypes (2)

C library is loaded into Python and accessed directly;

```python
#!/usr/bin/env python
from ctypes import *
hw_lib = CDLL('./hw.so')  # load shared library

hw_lib.hw1.restype = c_double  # specify return type
s = hw_lib.hw1(c_double(1), c_double(2.14159))
print(s, type(s))

# automatic conversion of arguments from Python to ctypes:
hw_lib.hw1.argtypes = [c_double, c_double]
s = hw_lib.hw1(1, 2.14159)
print(s, type(s))

hw_lib.hw2.argtypes = [c_double, c_double]
hw_lib.hw1.restype = None  # returns void
hw_lib.hw2(1, 2.14159)

s = c_double()
hw_lib.hw3(c_double(1), c_double(2.14159), byref(s))
print(s.value)
```

## Ctypes bottom line

* Works well for interfacing a few C functions
* Less convenient for a large number of function calls
* Can be wrapped in a Python module, but with a performance loss

## Remember: The best tool depends on the task
<center>![Mixed Programming Python tools](python_integration_tools.svg "Python")</center>